# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.train.automl import AutoMLConfig
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import azureml.core
from azureml.core import Workspace, Experiment
from azureml.core.compute import AmlCompute, ComputeTarget
from azureml.widgets import RunDetails
from azureml.core.dataset import Dataset
import os

Failure while loading azureml_run_type_providers. Failed to load entrypoint automl = azureml.train.automl.run:AutoMLRun._from_run_dto with exception (azureml-dataprep-rslex 1.2.3 (c:\users\long8\appdata\roaming\python\python38\site-packages), Requirement.parse('azureml-dataprep-rslex<1.10.0a,>=1.9.0dev0'), {'azureml-dataprep'}).


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
def compute_target(ws,compute_name='cpu-cluster-1',vm_size="STANDARD_D2_V2",compute_min_nodes=0,compute_max_nodes=4):
    if compute_name in ws.compute_targets:
        compute_target = ws.compute_targets[compute_name]
        if compute_target and type(compute_target) is AmlCompute:
            print('found compute target. just use it. ' + compute_name)
    else:
        print('Creating a new compute target')
        config = AmlCompute.provisioning_configuration(vm_size=vm_size,
                                                       min_nodes=compute_min_nodes,
                                                       max_nodes=compute_max_nodes)
        # create a cluster
        compute_target = ComputeTarget.create(ws,compute_name,config)
        compute_target.wait_for_completion(show_output=True)
    return compute_target

In [3]:
ws = Workspace.from_config('.')

# choose a name for experiment
experiment_name = 'project-automl-1'

experiment=Experiment(ws, experiment_name)

# create compute target
compute_target = compute_target(ws,compute_name='cpu-cluster',
                                vm_size="STANDARD_D2_V2",
                                compute_min_nodes=0,
                                compute_max_nodes=4)

If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.


found compute target. just use it. cpu-cluster


In [6]:
# load data set
workspace = Workspace.from_config('.')
dataset = Dataset.get_by_name(workspace, name='adult')

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [7]:
# TODO: Put your automl config here
automl_config = AutoMLConfig(
    training_data= dataset,
    validation_size = 0.2,
    label_column_name='class-label',
    experiment_timeout_minutes=30,
    compute_target=compute_target,
    task='classification',
    primary_metric='accuracy',
    n_cross_validations=3,
    enable_onnx_compatible_models=True,
    iterations=10)

In [8]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
project-automl-1,AutoML_6134171f-7acb-4c71-92b5-5f171357441d,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [9]:
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Experiment,Id,Type,Status,Details Page,Docs Page
project-automl-1,AutoML_6134171f-7acb-4c71-92b5-5f171357441d,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

******************************************************************

{'runId': 'AutoML_6134171f-7acb-4c71-92b5-5f171357441d',
 'target': 'cpu-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-04-03T05:28:43.036356Z',
 'endTimeUtc': '2021-04-03T05:51:38.536984Z',
 'properties': {'num_iterations': '10',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0.2',
  'acquisition_parameter': '0',
  'num_cross_validation': '3',
  'target': 'cpu-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"project-automl-1","subscription_id":"c8eb054b-91b3-425d-acbc-fe49c970c7c4","resource_group":"udacity","workspace_name":"azProject3","region":"australiaeast","compute_target":"cpu-cluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":false,"enable_run_restructure":false,"iterations":10,"primary_metric":"accuracy","task_type":"classification","data_script":null,"test_size":0.0,"validation_size":0.2,"n_cross_validations":3,"y

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [10]:
#TODO: Save the best model
best_run, model = remote_run.get_output()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']
print("Run ID:",best_run.id)
print('Metrics:',best_run_metrics['accuracy'])
print(best_run_metrics)

for metric_name in best_run_metrics:
    metric = best_run_metrics[metric_name]
    print(metric_name, metric)
    
model_name = 'model'
best_run.register_model(model_path='outputs/model.pkl', model_name=model_name)

Run ID: AutoML_6134171f-7acb-4c71-92b5-5f171357441d_9
Metrics: 0.8586744639376219
{'average_precision_score_weighted': 0.9274256651311404, 'f1_score_weighted': 0.8511145711639676, 'weighted_accuracy': 0.9120226063616871, 'f1_score_macro': 0.7960020419862558, 'AUC_micro': 0.9370878361433147, 'log_loss': 0.3276420805872589, 'norm_macro_recall': 0.5441401243663043, 'balanced_accuracy': 0.7720700621831522, 'accuracy': 0.8586744639376219, 'precision_score_weighted': 0.8542112725005474, 'AUC_macro': 0.9137353870217185, 'average_precision_score_micro': 0.9390397518842275, 'recall_score_weighted': 0.8586744639376219, 'precision_score_macro': 0.8374161855765948, 'matthews_correlation': 0.6059407028954847, 'f1_score_micro': 0.8586744639376219, 'average_precision_score_macro': 0.8889382241189946, 'AUC_weighted': 0.9137353870217185, 'recall_score_micro': 0.8586744639376219, 'precision_score_micro': 0.8586744639376219, 'recall_score_macro': 0.7720700621831522, 'accuracy_table': 'aml://artifactId/Ex

Model(workspace=Workspace.create(name='azProject3', subscription_id='c8eb054b-91b3-425d-acbc-fe49c970c7c4', resource_group='udacity'), name=model, id=model:1, version=1, tags={}, properties={})

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [11]:
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.compute import ComputeTarget
from azureml.core.webservice import AciWebservice
from azureml.core.model import InferenceConfig
from azureml.core.model import Model
from azureml.core import Environment
import sklearn
inference_config = InferenceConfig(entry_script='score.py')
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1,auth_enabled=True)
model = ws.models[model_name]
service = Model.deploy(workspace=ws,
                       name='automlclassifier',
                       models=[model],
                       inference_config=inference_config,
                       deployment_config=aci_config)
service.wait_for_deployment(show_output = True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-04-03 16:54:25+11:00 Creating Container Registry if not exists.
2021-04-03 16:54:27+11:00 Use the existing image.
2021-04-03 16:54:27+11:00 Generating deployment configuration.
2021-04-03 16:54:28+11:00 Submitting deployment to compute..
2021-04-03 16:54:34+11:00 Checking the status of deployment automlclassifier..
2021-04-03 16:55:12+11:00 Checking the status of inference endpoint automlclassifier.
Succeeded
ACI service creation operation finished, operation "Succeeded"


TODO: In the cell below, send a request to the web service you deployed to test it.

In [15]:
import urllib.request
import json
import os
import ssl

def allowSelfSignedHttps(allowed):
    # bypass the server certificate verification on client side
    if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context

allowSelfSignedHttps(True) # this line is needed if you use self-signed certificate in your scoring service.

data = {
    "data":
    [
        {
            'age': "21",
            'workclass': "Private",
            'fnlwgt': "287436",
            'education': " HS-grad",
            'education-num': "9",
            'marital-status': " Never-married",
            'occupation': " Machine-op-inspct",
            'relationship': " Husband",
            'race': " Black",
            'sex': " Male",
            'capital-gain': "0",
            'capital-loss': "0",
            'hours-per-week': "38",
            'native-country': "United-States",
        },
    ],
}


body = str.encode(json.dumps(data))

url = 'http://cc68f5c4-ea70-40c7-b67f-2f016ec72236.australiaeast.azurecontainer.io/score'
api_key = 'gKKfZXDrLRvDCMMcnytt61Qs9hnHk6Sb' # Replace this with the API key for the web service
headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key)}

req = urllib.request.Request(url, body, headers)

try:
    response = urllib.request.urlopen(req)

    result = response.read()
    print(result)
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))

    # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
    print(error.info())
    print(json.loads(error.read().decode("utf8", 'ignore')))


b'"{\\"result\\": [\\"Low\\"]}"'


TODO: In the cell below, print the logs of the web service and delete the service

In [16]:
print(service.get_logs())

2021-04-03T06:02:25,206565100+00:00 - iot-server/run 
2021-04-03T06:02:25,212742200+00:00 - nginx/run 
2021-04-03T06:02:25,223108300+00:00 - rsyslog/run 
2021-04-03T06:02:25,230220900+00:00 - gunicorn/run 
/usr/sbin/nginx: /azureml-envs/azureml_2b14f450572e78de640d54eaabed5e4d/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_2b14f450572e78de640d54eaabed5e4d/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_2b14f450572e78de640d54eaabed5e4d/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_2b14f450572e78de640d54eaabed5e4d/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_2b14f450572e78de640d54eaabed5e4d/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
rsyslogd

In [ ]:
compute_target.delete()